In [ ]:
#!pip install py2neo
#!pip install nlp_rake
#!pip install pytextrank
#!python3 -m pip install -U pip
#!python3 -m pip install -r requirements.txt
#!python3 -m spacy download en_core_web_sm

In [2]:
import pandas as pd
from py2neo import Graph,Node,Relationship
from py2neo.bulk import create_nodes
import re
from py2neo.bulk import create_relationships
import nltk
import re
import collections 

In [3]:
# Read Reddit post on wallstreetbets subreddit
df_p = pd.read_csv('archive/wsb-aug-2021-posts.csv', usecols=None,names=['type', 'p_id', 'subid',
                    'name', 'nsfw','p_created','p_permalink','domain','url','selftext','p_title','p_score'],skiprows=1)

In [4]:
# Retain relevant columns
df_p = df_p[['p_id','p_created','p_permalink','p_title','p_score']]
df_p.shape

(25751, 5)

In [5]:
# Read Comments to reddit posts file
df_c = pd.read_csv('archive/wsb-aug-2021-comments.csv', usecols=None, names=['type', 'c_id', 'subid',
                    'name', 'nsfw','c_created','c_permalink','c_body','c_sentiment','c_score'],skiprows=1)

In [6]:
# Retain relevant columns
df_c=df_c[['c_id','c_created','c_permalink','c_body','c_score']]
df_c['parentid']=df_c.c_permalink.str.slice(49,55) # Extract original post id from permalink
df_c.shape

(1001160, 6)

In [7]:
df_p.head()

,p_id,p_created,p_permalink,p_title,p_score
0,pfi0x7,1630454321,https://old.reddit.com/r/wallstreetbets/commen...,Is BABA the next?,1
1,pfhz92,1630454157,https://old.reddit.com/r/wallstreetbets/commen...,$TELL- According to Wall Street Journal its a ...,1
2,pfhxzc,1630454028,https://old.reddit.com/r/wallstreetbets/commen...,IS BABA next?,1
3,pfhw6s,1630453851,https://old.reddit.com/r/wallstreetbets/commen...,1.4K to 7.K overnight on FIVN puts. Thanks ZM!,79
4,pfhtyf,1630453627,https://old.reddit.com/r/wallstreetbets/commen...,1.4K to 7.1K overnight on FIVN puts. Thanks ZM!,2


In [8]:
df_c.head()

,c_id,c_created,c_permalink,c_body,c_score,parentid
0,hb4hdm3,1630454394,https://old.reddit.com/r/wallstreetbets/commen...,What's updog,3,pfdkjw
1,hb4hdm8,1630454394,https://old.reddit.com/r/wallstreetbets/commen...,Don’t tell em,1,pfdkjw
2,hb4hdjc,1630454393,https://old.reddit.com/r/wallstreetbets/commen...,"I realize this, ive been losing thousands shoo...",2,pfdkjw
3,hb4hdgo,1630454392,https://old.reddit.com/r/wallstreetbets/commen...,then it tanks after earnings,4,pfgr1h
4,hb4hdeh,1630454391,https://old.reddit.com/r/wallstreetbets/commen...,Are you saying I should or shouldn’t yolo my l...,1,pf3xee


### Data Cleanup and merging

In [9]:
df_p.shape, df_c.shape

((25751, 5), (1001160, 6))

In [10]:
#Dropping null values
df_c.dropna(inplace=True)

In [11]:
df_p.shape, df_c.shape

((25751, 5), (1001158, 6))

In [12]:
# Dropping rows with body and title as [removed]
df_p=df_p[~df_p.p_title.str.match(pat='\[?removed\]')]
df_c=df_c[~df_c.c_body.str.match(pat='\[?removed\]')]
df_p.shape,df_c.shape

((25751, 5), (857417, 6))

In [13]:
# Dropping rows with body and title as [deleted]
df_p=df_p[~df_p.p_title.str.match(pat='\[?deleted\]')]
df_c=df_c[~df_c.c_body.str.match(pat='\[?deleted\]')]
df_p.shape,df_c.shape

((25751, 5), (813950, 6))

In [14]:
# Dropping rows with body and title as [deleted]
df_p=df_p[~df_p.p_title.str.contains(pat='Your submission was removed')]
df_c=df_c[~df_c.c_body.str.contains(pat='Your submission was removed')]
df_p.shape,df_c.shape

((25751, 5), (798002, 6))

In [15]:
# Dropping rows with body and title as [deleted]
df_p=df_p[~df_p.p_title.str.contains('I am a bot from')]
df_c=df_c[~df_c.c_body.str.match('I am a bot from')]
df_p.shape,df_c.shape

((25751, 5), (796535, 6))

In [16]:
#Merge the dataframes
df_m=df_p.merge(df_c, left_on='p_id', right_on='parentid')

In [17]:
df_m.shape

(782307, 11)

In [18]:
df_p.reset_index(inplace=True,drop=True)
df_c.reset_index(inplace=True,drop=True)
df_m.reset_index(inplace=True,drop=True)
df_m.head()

,p_id,p_created,p_permalink,p_title,p_score,c_id,c_created,c_permalink,c_body,c_score,parentid
0,pfhw6s,1630453851,https://old.reddit.com/r/wallstreetbets/commen...,1.4K to 7.K overnight on FIVN puts. Thanks ZM!,79,hb4grlq,1630454109,https://old.reddit.com/r/wallstreetbets/commen...,How do people do this?,4,pfhw6s
1,pfhw6s,1630453851,https://old.reddit.com/r/wallstreetbets/commen...,1.4K to 7.K overnight on FIVN puts. Thanks ZM!,79,hb4ga59,1630453885,https://old.reddit.com/r/wallstreetbets/commen...,\n**User Report**| | | |\n:--|:--|:--|:--\n**T...,1,pfhw6s
2,pfhtyf,1630453627,https://old.reddit.com/r/wallstreetbets/commen...,1.4K to 7.1K overnight on FIVN puts. Thanks ZM!,2,hb4ft1v,1630453666,https://old.reddit.com/r/wallstreetbets/commen...,\n**User Report**| | | |\n:--|:--|:--|:--\n**T...,1,pfhtyf
3,pfhq3j,1630453246,https://old.reddit.com/r/wallstreetbets/commen...,Does anyone know what the first stock symbol i...,4,hb4gzf2,1630454210,https://old.reddit.com/r/wallstreetbets/commen...,"That company was delisted, the value is prob a...",3,pfhq3j
4,pfhq3j,1630453246,https://old.reddit.com/r/wallstreetbets/commen...,Does anyone know what the first stock symbol i...,4,hb4g8h8,1630453863,https://old.reddit.com/r/wallstreetbets/commen...,definitely means you’re gay bro.,6,pfhq3j


### Most popular Post in Dataset

In [19]:
print(f'''Most popular posts is {df_p[df_p.p_score == df_p.p_score.max()].p_title.values} 
with the score of {df_p.p_score.max()}''')

Most popular posts is ['My portfolio after discovering wsb'] 
with the score of 45414


### Most popular Comment in Dataset

In [20]:
print(f'''Most popular posts is {df_c[df_c.c_score == df_c.c_score.max()].c_body.values} 
      with the score of {df_c.c_score.max()}''')

Most popular posts is ['Ok so I\'d just remove the letter "k" from the post because if you do that then you\'ll only be down $15 instead of $15k, and that\'s a really manageable loss.'] 
      with the score of 21129


### Most commented post in Dataset

In [21]:
a=df_m['parentid'].mode()
title=df_m.p_title[df_m.p_id == 'p3sv76']
print(f'Most commented post is: {title[title.index[0]]}')

Most commented post is: Weekend Discussion Thread for the Weekend of August 13, 2021


## Keyword Extraction

In [22]:
# We had a problem of scale. Running any line below was not possible for 1M+ records so we decided to curtail our dataset

# v = df_m.p_id.value_counts()
# df_m=df_m[df_m.p_id.isin(v.index[v.gt(12000)])]
# df_m.reset_index(drop=True, inplace=True)

In [23]:
print('Number of Posts:', len(df_m.p_id.unique()))
print('Number of Comments:', len(df_m.c_id.unique()))

Number of Posts: 12234
Number of Comments: 782307


### Using RAKE (We decided to use rack over spacy for better output and performance)

In [24]:
#Download stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/asachan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [25]:
from nlp_rake import Rake
r = Rake(
    min_chars=2,
    max_words=5,
    min_freq=1
)

In [26]:
posts = df_m.p_title.unique()
titles = df_m.c_body.values
print('Number of unique posts: ',len(posts))
print('Number of unique titles: ',len(titles))

Number of unique posts:  11598
Number of unique titles:  782307


In [27]:
# # Keyword extraction is failing for the dataset. Reducing the number of comments.
# b=[]
# for t in titles:
#     if len(t) > 100:
#         b.append(t)
# len(b)

In [28]:
%%time
# Get all stock symbol from title of the post.
title_ent=[]
for i in range(len(posts)):   
    keywords = r.apply(posts[i])
    if len(keywords) > 0:
        result = re.search("\$(\w+)", keywords[0][0])
        if result:
            title_ent.append(result.group(1))
print('Number of keywords extracted:', len(title_ent))

Number of keywords extracted: 1119
CPU times: user 1min 5s, sys: 322 ms, total: 1min 5s
Wall time: 1min 5s


In [29]:
%%time
# Get all stock symbol from the body of the comments.
body_ent=[]
for i in range(len(titles)):    
    keywords = r.apply(titles[i])
    if len(keywords) > 0:
        for i in range(len(keywords)):
            result = re.search("\$(\w+)", keywords[i][0])
            if result and result.group(1).isalpha():
                body_ent.append(result.group(1))
print('Number of keywords extracted:', len(body_ent))

Number of keywords extracted: 11441
CPU times: user 1h 4min 38s, sys: 17.9 s, total: 1h 4min 56s
Wall time: 1h 5min 1s


In [30]:
title_ent=set(title_ent)
body_ent=set(body_ent)
m_com=title_ent.union(body_ent)
m_com = list(body_ent)
print('Number of companies mentioned in posts',len(title_ent))
print('Number of companies mentioned in comments',len(body_ent))

print('Total companies mentioned across posts and comments', len(m_com))
print('Sample of companies stock symbol extracted', m_com[100:150])

Number of companies mentioned in posts 440
Number of companies mentioned in comments 1421
Total companies mentioned across posts and comments 1421
Sample of companies stock symbol extracted ['bynd', 'stinkybedsheets', 'slv', 'chd', 'cmg', 'dicc', 'ha', 'cock', 'vixy', 'cwh', 'ree', 'fud', 'bbae', 'aeo', 'on', 'oxy', 'm', 'jepi', 'gush', 'brzl', 'pain', 'mile', 'dia', 'xone', 'vmeo', 'siri', 'goos', 'pbya', 'dm', 'vaseline', 'cms', 'rdsa', 'mmnd', 'gummybears', 'dg', 'job', 'prpl', 'go', 'apha', 'dal', 'bag', 'syf', 'ctxr', 'swch', 'pelosi', 'tvix', 'poon', 'mcfe', 'ui', 'rooting']


### Using pytextrank and spacy (Output especially with '$' prefixed string is not as good as RACK)

In [31]:
#https://towardsdatascience.com/keyword-extraction-a-benchmark-of-7-algorithms-in-python-8a905326d93f - Keyword extraction comparison
# import pytextrank
# import spacy

In [32]:
# nlp = spacy.load("en_core_web_sm")
# nlp.add_pipe("textrank")

In [33]:
# %%time
# ## Extract keywords from title of the post
# p_ent=[]
# for i in range(len(df_p)):
#     doc=nlp(df_p.p_title[i])
#     for ent in doc.ents:
#         p_ent.append(ent.text) # named entity and noun chunk yields same result

In [34]:
# %%time
# ## Extract keywords from body of the comments
# c_ent=[]
# for i in range(len(df_c)):
#     doc=nlp(df_c.c_body[i])
#     for ent in doc.ents:
#         c_ent.append(ent.text) # named entity and noun chunk yields same result

### Validating and getting stock price from finance.yahoo.com

In [35]:
%%time
import yfinance as yf
comp_prices = pd.DataFrame()
for comp in m_com:
    tick = yf.Ticker(comp)
    if tick:
        dat = tick.history(start='2021-08-01', end='2021-08-31')
        dat['Delta'] = dat['Close']-dat['Open']
        dat['Ticker'] = comp
        comp_prices = comp_prices.append(dat)
comp_prices = comp_prices.reset_index(drop=False)
df_prices=comp_prices[['Ticker','Date','Open','Close','Delta']]

- XXX: No data found for this date range, symbol may be delisted
- CCP: No data found for this date range, symbol may be delisted
- MARGIN: No data found, symbol may be delisted
- WHITEY: No data found, symbol may be delisted
- CK: No data found for this date range, symbol may be delisted
- PROBE: No data found, symbol may be delisted
- CUMT: No data found, symbol may be delisted
- YOUS: No data found, symbol may be delisted
- SCHMECKIE: No data found, symbol may be delisted
- HOLE: No data found, symbol may be delisted
- NUTT: No data found for this date range, symbol may be delisted
- PNIS: No data found, symbol may be delisted
- CHOWN: No data found, symbol may be delisted
- APPLE: No data found, symbol may be delisted
- ARSENAL: No data found, symbol may be delisted
- KRSK: No data found, symbol may be delisted
- BLACKBERRY: No data found, symbol may be delisted
- ICU: No data found for this date range, symbol may be delisted
- INTCIS: No data found, symbol may be delisted
- CGN: N

In [36]:
df_prices.head()

,Ticker,Date,Open,Close,Delta
0,lcid,2021-08-02,24.193001,23.770000,-0.423
1,lcid,2021-08-03,23.930000,23.580000,-0.350
2,lcid,2021-08-04,23.480000,22.629999,-0.850
3,lcid,2021-08-05,22.920000,22.750000,-0.170
4,lcid,2021-08-06,22.934999,23.379999,0.445


In [37]:
print('Number of companies after validation: ', len(df_prices.Ticker.unique()))
comps=df_prices.Ticker.unique()

Number of companies after validation:  977


## Graph Section

#### Project and dabase was manually created

In [38]:
#g = Graph("bolt://localhost:11006", auth=('neo4j', 'admin'))
g = Graph("bolt://localhost:11005", auth=('new', 'admin'))

In [39]:
# # Create Post nodes
# data=[]
# posts=df_p
# for i in range(len(posts)):
#     x=posts.id[i]
#     y=posts.title[i]
#     z=str(posts.score[i])
#     dict1 = {'id':x, 'title':y, 'score':z}
#     data.append(dict1)
# create_nodes(g.auto(), data, labels={"Post"})

# #Create comments nodes
# data=[]
# comments=df_c
# for i in range(len(comments)):
#     x=comments.id[i]
#     y=comments.body[i]
#     z=str(comments.score[i])
#     zz=comments.parentid[i]
#     dict1 = {'id':x, 'title':y, 'score':z,'parentid':zz}
#     data.append(dict1)

# from itertools import islice
# stream = iter(data)
# batch_size = 10000
# while True:
#     batch = islice(stream, batch_size)
#     if batch:
#         create_nodes(g.auto(), batch, labels={"Comment"})
#     else:
#         break


In [47]:
df_m.shape

(782307, 11)

In [53]:
# more cleanup to avoid special character issues.

df_m=df_m[~df_m.c_body.str.contains('User Report')]
df_m.reset_index(drop=True,inplace=True)
df_m.shape

(775424, 11)

In [62]:
%%time
# This code creates nodes and relationship between nodes
for i in range(len(df_m)):

    a = Node("POST", p_id=df_m.p_id[i], 
             title=df_m.p_title[i], 
             score=str(df_m.p_score[i]), 
             created=str(df_m.p_created[i]))
    a.__primarylabel__ = "POST"
    a.__primarykey__ = "p_id"
    b = Node("COMMENT", c_id=df_m.c_id[i], 
             body=df_m.c_body[i], 
             score=str(df_m.c_score[i]), 
             parentid=df_m.parentid[i], 
             created=str(df_m.c_created[i]))
    b.__primarylabel__ = "COMMENT"
    b.__primarykey__ = "c_id"
    HAS = Relationship.type("HAS")
    g.merge(HAS(a, b)) # Relations between post and comment
    
    
# Matching company mentioned in post's title with extracted and validated company list
    
    keywords = r.apply(df_m.p_title[i])
    p_com=[]
     
    if len(keywords) > 0:
        for i in range(len(keywords)):
            result = re.search("\$(\w+)", keywords[i][0])
            if result and result.group(1).isalpha():
                p_com.append(result.group(1))
                
        for k in p_com:
            tt=[]
            for x in k.split():
                if x in comps:            
                    c = Node("COMPANY", name=x)
                    c.__primarylabel__ = "COMPANY"
                    c.__primarykey__ = "name"
                    MENTIONS = Relationship.type("MENTIONS")
                    g.merge(MENTIONS(a, c)) # Relations between comment and companies mentioned
                    
                    sp=df_prices[df_prices.Ticker==x]
                    sp.reset_index(drop=True,inplace=True)
                    for j in range(len(sp)):
                        d = Node("STOCKPRICE", created=str(sp.Date[j]), 
                                 sopen=str(sp.Open[j]), 
                                 sclose=str(sp.Close[j]), 
                                 diff=str(sp.Delta[j]))
                        d.__primarylabel__ = "STOCKPRICE"
                        d.__primarykey__ = "created"
                        ACTIVITY = Relationship.type("ACTIVITY")
                        g.merge(ACTIVITY(c, d)) # Relations between company and price movement

# Matching company mentioned in comment's body with extracted and validated company list
    
    keywords = r.apply(df_m.c_body[i])
    m_com=[]
     
    if len(keywords) > 0:
        for i in range(len(keywords)):
            result = re.search("\$(\w+)", keywords[i][0])
            if result and result.group(1).isalpha():
                m_com.append(result.group(1))

        for i in m_com:
            tt=[]
            for x in i.split():
                if x in comps:            
                    c = Node("COMPANY", name=x)
                    c.__primarylabel__ = "COMPANY"
                    c.__primarykey__ = "name"
                    MENTIONS = Relationship.type("MENTIONS")
                    g.merge(MENTIONS(b, c)) # Relations between comment and companies mentioned
                    
                    sp=df_prices[df_prices.Ticker==x]
                    sp.reset_index(drop=True,inplace=True)
                    for j in range(len(sp)):
                        d = Node("STOCKPRICE", created=str(sp.Date[j]), 
                                 sopen=str(sp.Open[j]), 
                                 sclose=str(sp.Close[j]), 
                                 diff=str(sp.Delta[j]))
                        d.__primarylabel__ = "STOCKPRICE"
                        d.__primarykey__ = "created"
                        ACTIVITY = Relationship.type("ACTIVITY")
                        g.merge(ACTIVITY(c, d)) # Relations between company and price movement

KeyboardInterrupt: 

In [61]:
g.delete_all()

In [ ]:
g.nodes.match("POST").count()

In [ ]:
g.nodes.match("COMMENT").count()

## Graph Queries

### Most popular Post

In [ ]:
g.run("match (p:POST)  return p.p_id, p.title,p.score order by p.score desc limit 1")

### Most popular Comment

In [ ]:
g.run("match (p:COMMENT) return p.c_id,p.body, p.score order by p.score desc limit 1")

### Most Commented Post

In [ ]:
g.run("match (p:POST) -[e:HAS] -> (c:COMMENT) return p.p_id,p.title, count(e) as num order by num desc limit 1")

### Most mentioned company

In [ ]:
g.run("match (c:COMMENT) -[e:MENTIONS] -> (com:COMPANY) return com.name as Name, count(e) as num order by num desc limit 1")

In [ ]:
## Demo ids
p_id: pfgfbf